## Pop & Housing variables pulled from 2010, and 2020 Censuses (using Cen API) for urban villages,  City of Phoenix and U.S.

Total Housing Units, Population, Population by Ethncity, Total Occupied Housing Units,  


In [1]:
import pandas as pd
import numpy as np

In [2]:
from geo import *
import get_dec as get

In [3]:
bgp_10 = pd.read_csv('../data/geo/bgp_vil_10.csv')
bgp_20 = pd.read_csv('../data/geo/bgp_vil_20.csv')
for df in [bgp_10,bgp_20]: df.geoid = df.geoid.apply(lambda x: '{0:0>12}'.format(x))
    
#get ride of area & geo stuff not being useed
bgp_20 = bgp_20.drop(['aland20','awater20','lat20','lon20','land_acre'],axis=1)
bgp_10 = bgp_10.drop(['aland10','awater10','lat10','lon10','land_acre'],axis=1)

In [4]:
source_dec = 'dec/sf1'
source_red = 'dec/pl'
years = {'2010':'P001001,P005003,P005004,P005005,P005006,P005007,P005008,P005009,P005010,H003001,H003002,H003003',\
        '2020':'P1_001N,P2_002N,P2_005N,P2_006N,P2_007N,P2_008N,P2_009N,P2_010N,P2_011N,H1_001N,H1_002N,H1_003N'}

col_10_rename={'P001001':'Pop_10E','P005003':'P_Wh_10E','P005004':'P_Bl_10E','P005006':'P_As_10E','P005010':'P_Hi_10E',\
               'P005005':'P_Ot_10E','P005007':'P_Ot_10E','P005008':'P_Ot_10E','P005009':'P_Ot_10E',\
              'H003001':'Hou_10E','H003002':'Hou_O_10E','H003003':'Hou_V_10E'}

col_20_rename = {'P1_001N':'Pop_20E','P2_002N':'P_Hi_20E',\
              'P2_005N':'P_Wh_20E','P2_006N':'P_Bl_20E',\
              'P2_007N':'P_Ot_20E','P2_008N':'P_As_20E','P2_009N':'P_Ot_20E',\
              'P2_010N':'P_Ot_20E','P2_011N':'P_Ot_20E','H1_001N':'Hou_20E',\
              'H1_002N':'Hou_O_20E','H1_003N':'Hou_V_20E'}

### Urban Village for 2010 and 2020

In [5]:
df10 = get.get_bgp(source_dec,list(years.keys())[0],years.get(list(years.keys())[0]))
df20 = get.get_bgp(source_red,list(years.keys())[1],years.get(list(years.keys())[1]))

In [6]:
def make_uvil(geodf,df,rename_col):
    df.rename(columns=rename_col,inplace=True)
    for col in df.columns[:-1]: df[col] = df[col].astype(int)
    df = pd.merge(geodf,df,how='left',left_on='geoid',right_on='GEO_ID')
    df = df.drop(['GEO_ID','geoid'],axis=1)
    df = df.groupby(df.columns,axis=1).sum().groupby('name').sum().reset_index()
    return df

In [7]:
u10 = make_uvil(bgp_10,df10,col_10_rename)

In [8]:
u10.head(3)

,name,Hou_10E,Hou_O_10E,Hou_V_10E,P_As_10E,P_Bl_10E,P_Hi_10E,P_Ot_10E,P_Wh_10E,Pop_10E
0,Ahwatukee Foothills,33528,31300,2228,4693,3969,9554,3169,55959,77344
1,Alhambra,52066,43584,8482,4774,7687,61060,5755,47168,126444
2,Camelback East,69263,58921,10342,2858,7452,42558,6609,72676,132153


In [9]:
u20 = make_uvil(bgp_20,df20,col_20_rename)

In [10]:
u20.head(3)

,name,Hou_20E,Hou_O_20E,Hou_V_20E,P_As_20E,P_Bl_20E,P_Hi_20E,P_Ot_20E,P_Wh_20E,Pop_20E
0,Ahwatukee Foothills,34527,32555,1972,5389,4876,12864,6270,51205,80604
1,Alhambra,52997,48670,4327,7068,13457,68897,9290,46170,144882
2,Camelback East,75182,67196,7986,4242,9134,42702,10205,78666,144949


In [11]:
uvil = pd.merge(u10,u20,how='left',on='name')

### Phoenix for 2010 and 2020

In [12]:
def make_table(df,rename):
    df.rename(columns=rename,inplace=True)
    for col in df.columns[:-1]: df[col] = df[col].astype(int)
    df = df.groupby(df.columns,axis=1).sum()
    return df

In [13]:
p10 = get.get_phx(source_dec,list(years.keys())[0],years.get(list(years.keys())[0]))
p20 = get.get_phx(source_red,list(years.keys())[1],years.get(list(years.keys())[1]))

In [14]:
p10 = make_table(p10,col_10_rename)
p20 = make_table(p20,col_20_rename)

In [15]:
phx = pd.merge(p10,p20,how='left',on='GEO_ID')
phx.rename(columns={'GEO_ID':'name'},inplace=True)

## U.S. for 2010 and 2020

In [16]:
us10 = get.get_us(source_dec,list(years.keys())[0],years.get(list(years.keys())[0]))
us20 = get.get_us(source_red,list(years.keys())[1],years.get(list(years.keys())[1]))

In [17]:
us10 = make_table(us10,col_10_rename)
us20 = make_table(us20,col_20_rename)

In [18]:
us = pd.merge(us10,us20,how='left',on='us')

In [19]:
us.rename(columns={'us':'name'},inplace=True)
us = us[['name']+[col for col in us.columns if col !='name']]

## Concat all geos together & export to excel

In [20]:
final = pd.concat([uvil,phx,us])

In [21]:
final

,name,Hou_10E,Hou_O_10E,Hou_V_10E,P_As_10E,P_Bl_10E,P_Hi_10E,P_Ot_10E,P_Wh_10E,Pop_10E,Hou_20E,Hou_O_20E,Hou_V_20E,P_As_20E,P_Bl_20E,P_Hi_20E,P_Ot_20E,P_Wh_20E,Pop_20E
0,Ahwatukee Foothills,33528,31300,2228,4693,3969,9554,3169,55959,77344,34527,32555,1972,5389,4876,12864,6270,51205,80604
1,Alhambra,52066,43584,8482,4774,7687,61060,5755,47168,126444,52997,48670,4327,7068,13457,68897,9290,46170,144882
2,Camelback East,69263,58921,10342,2858,7452,42558,6609,72676,132153,75182,67196,7986,4242,9134,42702,10205,78666,144949
3,Central City,23571,19005,4566,714,5315,38963,2143,12124,59259,26537,22983,3554,1189,7519,33689,3431,15624,61452
4,Deer Valley,71285,64798,6487,7201,4804,27543,5720,122034,167302,76899,72641,4258,11726,7773,35544,11568,119178,185789
5,Desert View,23188,20341,2847,2293,728,3583,1237,43976,51817,29184,26089,3095,4870,1099,5128,3315,50595,65007
6,Encanto,28214,23108,5106,1650,3138,25337,2966,22843,55934,30714,27399,3315,1925,3983,24112,4252,25743,60015
7,Estrella,23595,20360,3235,1374,6253,61336,2235,13302,84500,25571,23944,1627,1446,8138,70334,3466,12741,96125
8,Laveen,15884,13979,1905,3183,8073,22141,2007,12430,47834,18677,18136,541,3102,12276,33689,4064,12916,66047
9,Maryvale,66024,56392,9632,2926,11177,158903,4971,32375,210352,67598,64016,3582,3910,15472,173614,8037,28463,229496


In [22]:
final.to_excel('output/Dec_pop_hou_race.xlsx',index=False)